<a href="https://colab.research.google.com/github/rvraghvender/DeepLearningProjects/blob/main/ConvolutionNeuralNetworks/ResidualNetworks/Residual_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Residual Networks

Here in this project, we have to build a very deep convolution network, using Residual Networks (ResNets). In theory, very deep neural network can represent very complex functions; but in practice, they are hard to train. ResNets introduced by [He et al.](https://arxiv.org/pdf/1512.03385.pdf), allow you to train much deeper networks than were previously feasible.

In this project, we will be
-    Implement the basic blocks of ResNets in a deep neural network using Keras
-    Put together these building blocks to implement and train a state-of-the-art neural network for image classification
-    Implement a skip connection in our network

In [2]:
# Importing Relevant packages
import tensorflow as tf
import  numpy as np
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPool2D
from tensorflow.keras.models import Model, load_model
from resnets_utils import *
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow

from test_utils import summary, comparator
import public_tests

%matplotlib inline

## The problem of Very Deep Neural Networks

In recent years, neural networks have become much deeper, with the state-of-the-art networks evolving from having just few layers (e.g. AlexNet) to over a hundred layers.

-    The main benefit of a very deep network is that it can represent very complex functions. It can also learn features at many different levels of abstraction, from edges (at the shallow layers, cloer to the input) to very complex features (at the deeper layers, closer to the output).

-   However, using a deeper neural network doesn't always help. A huge barrier to train them is vanishing gradients: very deep neural networks often have a gradient signal that goes to zero quickly, thus making gradient descent prohibitively slow.

-   More specifically, during gradient descent, as we backpropagate from the final layer back to the first layer, we are multiplying by the weight matri on each step, and thus the gradient can descrease exponentially quickly to zero (or, in rare cases, grow exponentially quickly and "explode" from gaining very large values).

-    During training, we might therefore see the magnitude (or norm) of the gradient for the shallower layer decrease to zero very rapidly as training proceeds.



<img src="vanishing_grad_kiank.png" style="width:450px;height:220px;">
<caption><center> <u> <font color='purple'> <b>Figure 1</b> </u><font color='purple'>  : <b>Vanishing gradient</b> <br> The speed of learning decreases very rapidly for the shallower layers as the network trains </center></caption>

Not to worry! We can solve this problem by building a Residual Network!


## Building a Residual Network
In ResNets, a "shortcut" or a "skip connection" allows the model to skip layers:

<img src="skip_connection_kiank.png" style="width:650px;height:200px;">
<caption><center> <u> <font color='purple'> <b>Figure 2</b> </u><font color='purple'>  : A ResNet block showing a skip-connection <br> </center></caption>